In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../data/cleaned_data.csv")

Una consideración importante que debemos tener en cuenta con este código es que las ganancias no pueden compararse directamente entre años distintos, ya que no mantienen la misma magnitud económica. Por lo tanto, es necesario considerar la inflación y la devaluación del dinero.
Para garantizar la comparabilidad, convertimos todos los valores correspondientes a años anteriores a 2024 en su valor presente.

In [ ]:
factores_inflacion = {
    2024: 1.00,       # Año Base
    2023: 1.0928,     # Inflación 2023 (9.28%)
    2022: 1.2362,     # Inflación 2023 + 2022 (Acumulada ~23%)
    2021: 1.3057      # Inflación 2023 + 2022 + 2021 (Acumulada ~30%)
}

# Crear columna de factor
df['Factor_Ajuste'] = df['Año de Corte'].map(factores_inflacion)

# Aplicar la corrección a tus columnas monetarias
columnas_dinero = ['INGRESOS OPERACIONALES', 'TOTAL ACTIVOS', 'GANANCIA (PÉRDIDA)',"TOTAL PATRIMONIO","TOTAL PASIVOS"]

for col in columnas_dinero:
    # Creamos la versión "REAL" de la columna
    nombre_real = col + '_REAL'
    df[nombre_real] = df[col] * df['Factor_Ajuste']

In [ ]:
for i in df.columns:
    print(f"La columna {i} tiene un total de {len(df[i].value_counts())} valores unicos")

In [ ]:
df["ROA"]=df["GANANCIA (PÉRDIDA)_REAL"]/df["TOTAL ACTIVOS_REAL"] # Mide la eficiencia con la que una empresa utiliza sus activos para generar ganancias
df["Endeudamiento"] = df["TOTAL PASIVOS_REAL"]/df["TOTAL ACTIVOS_REAL"] # Mide el porcentaje del negocio financiado con deuda
df["Rotacion activos"] = df["INGRESOS OPERACIONALES_REAL"]/df["TOTAL ACTIVOS_REAL"] # Mide qué tan bien la empresa utiliza sus activos para generar ingresos

In [ ]:
df_numericas = df.select_dtypes(include=[np.number])
df_numericas.info()

Al calcular los indicadores financieros identificamos la presencia de valores nulos e infinitos. Esto ocurre porque algunas empresas registran activos o pasivos iguales a cero, lo que provoca divisiones por cero y, en consecuencia, errores matemáticos que generan valores infinitos.

Desde una perspectiva de negocio, es importante señalar que una empresa no puede operar con activos iguales a cero. Esta situación podría deberse a que los datos están expresados en magnitudes de billones de pesos, por lo que valores muy pequeños podrían aproximarse a cero durante el registro o procesamiento.

Teniendo en cuenta lo anterior, se propone definir un valor mínimo para las empresas que presentan activos igual a 0, asignándoles un valor de 0,01. De esta manera, es posible conservar registros que pueden ser útiles para el modelo y, al mismo tiempo, evitar la sobreestimación de ciertos indicadores financieros.

In [ ]:
VALOR_SUELO = 0.01 
mask_activos_zero = df['TOTAL ACTIVOS'] == 0
df.loc[mask_activos_zero, 'TOTAL ACTIVOS'] = VALOR_SUELO

In [ ]:
df["ROA"]=df["GANANCIA (PÉRDIDA)"]/df["TOTAL ACTIVOS"] # Mide la eficiencia con la que una empresa utiliza sus activos para generar ganancias 
df["Endeudamiento"] = df["TOTAL PASIVOS"]/df["TOTAL ACTIVOS"] # Mide el porcentaje del negocio financiado con deuda
df["Rotacion activos"] = df["INGRESOS OPERACIONALES"]/df["TOTAL ACTIVOS"] # Mide qué tan bien la empresa utiliza sus activos para generar ingresos

No usaremos el indicador ROA para entrenar el modelo debido a que requiere de la variable objetivo para su calculo, esto estaría entrenando el modelo con la respuesta.

In [ ]:
df_numeric = df.select_dtypes(include=[np.number])
matrix_corr=df_numericas.corr("spearman")
plt.figure(figsize=(16, 12))
sns.heatmap(data=matrix_corr, cmap="coolwarm", annot=True, fmt=".2f")
plt.show()


La columna CIIU aporta un valor significativo, incluso superior al de la columna Macrosector, ya que esta última es demasiado general. No obstante, la columna CIIU presenta inicialmente una alta cardinalidad (418 valores únicos), lo que podría ralentizar el modelo. Por este motivo, como estrategia de procesamiento, considero adecuado agrupar aquellos códigos CIIU que aparecen menos de 450 veces en el dataset. Con esta aproximación se conserva la información más relevante, manteniendo los 20 códigos CIIU más frecuentes y agrupando los demás en una categoría denominada “OTROS”.

In [ ]:
a = df["CIIU"].value_counts().sort_values()
resultado = []

for i in a:
    if i > 450:
        resultado.append(i)

print(resultado)
print(len(resultado))

In [ ]:
top_20_ciiu = df["CIIU"].value_counts().head(20).index

df.loc[~df["CIIU"].isin(top_20_ciiu), "CIIU"] = "Otros"

print(df["CIIU"].value_counts())

In [ ]:
columnas_seleccion =["SUPERVISOR","REGIÓN","DEPARTAMENTO DOMICILIO","CIIU","MACROSECTOR","TOTAL ACTIVOS_REAL","Endeudamiento","Rotacion activos","Año de Corte","GANANCIA (PÉRDIDA)_REAL"]
mode_data = df[columnas_seleccion]

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(15, 20))

sns.histplot(
    data=mode_data[mode_data["Año de Corte"] == 2021],
    x="TOTAL ACTIVOS_REAL", 
    palette="Set2",
    ax=ax[0],
    kde=True,
    bins=400
)
ax[0].set_title("Distribución de ganancias por año 2021", fontsize=14, fontweight='bold')
ax[0].tick_params(axis="x", rotation=45)

sns.histplot(
    data=mode_data[mode_data["Año de Corte"] == 2022],
    x="TOTAL ACTIVOS_REAL", 
    palette="Set2",
    ax=ax[1],
    kde=True,
    bins=400
)
ax[1].set_title("Distribución de ganancias por año 2022", fontsize=14, fontweight='bold')
ax[1].tick_params(axis="x", rotation=45)

sns.histplot(
    data=mode_data[mode_data["Año de Corte"] == 2023],
    x="TOTAL ACTIVOS_REAL", 
    palette="Set2",
    ax=ax[2],
    kde=True,
    bins=400
)
ax[2].set_title("Distribución de ganancias por año 2023", fontsize=14, fontweight='bold')
ax[2].tick_params(axis="x", rotation=45)

sns.histplot(
    data=mode_data[mode_data["Año de Corte"] == 2024],
    x="TOTAL ACTIVOS_REAL", 
    palette="Set3",
    ax=ax[3],
    kde=True,
    bins=400
)
ax[3].set_title("Distribución de ganancias por año 2024", fontsize=14, fontweight='bold')
ax[3].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline


# 1. Definir columnas
cols_log_scale = ['TOTAL ACTIVOS']  # Log + Robust
cols_robust_only = ['Rotacion', 'Apalancamiento'] # Solo Robust
cols_categorical = ['MACROSECTOR', 'SUPERVISOR', 'CIIU_agrupado'] # OneHot

# 2. Construir transformadores
preprocessing = ColumnTransformer(
    transformers=[
        # A. Activos: Logaritmo primero, luego RobustScaler
        ('log_robust', Pipeline([
            ('log', FunctionTransformer(np.log1p, validate=False)),
            ('scaler', RobustScaler())
        ]), cols_log_scale),

        # B. Ratios: Solo RobustScaler
        ('robust', RobustScaler(), cols_robust_only),

        # C. Categóricas: OneHotEncoding
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cols_categorical)
    ],
    remainder='drop' # Ignorar cualquier otra columna que no esté en la lista
)

# Este objeto 'preprocessing' es el que meterás en tu Pipeline final con el modelo.